# OpenNMT-py によるニューラル機械翻訳演習
OpenNMT-py というツールを用いて英日機械翻訳の演習を行います。
Google Colaboratory は GPU を用いた実験ができるので、GPU を用いてニューラル機械翻訳の実験をしましょう。（「編集」→「ノートブックの設定」から「ハードウェアアクセラレータ」を GPU にしておいてください。以下、Python 3 で説明します。）
データは田中コーパスという自由に使える対訳コーパスを使用します。翻訳結果を実際に見て評価する、ということも演習の目的です。

# OpenNMT-py の設定

## ダウンロード


In [1]:
# !wget https://github.com/OpenNMT/OpenNMT-py/archive/master.zip

In [2]:
# !unzip master.zip

In [3]:
# cd OpenNMT-py-master

## インストール

In [4]:
# !pip install OpenNMT-py

In [5]:
%%bash
cd /content
wget https://github.com/OpenNMT/OpenNMT-py/archive/1.2.0.zip
unzip 1.2.0.zip
cd OpenNMT-py-1.2.0
python setup.py install

Archive:  1.2.0.zip
60125c807d1cb18099a69dbfba699bcdf30560b1
   creating: OpenNMT-py-1.2.0/
  inflating: OpenNMT-py-1.2.0/.gitignore  
  inflating: OpenNMT-py-1.2.0/.travis.yml  
  inflating: OpenNMT-py-1.2.0/CHANGELOG.md  
  inflating: OpenNMT-py-1.2.0/CONTRIBUTING.md  
  inflating: OpenNMT-py-1.2.0/LICENSE.md  
  inflating: OpenNMT-py-1.2.0/README.md  
   creating: OpenNMT-py-1.2.0/available_models/
  inflating: OpenNMT-py-1.2.0/available_models/example.conf.json  
   creating: OpenNMT-py-1.2.0/config/
  inflating: OpenNMT-py-1.2.0/config/config-rnn-summarization.yml  
  inflating: OpenNMT-py-1.2.0/config/config-transformer-base-1GPU.yml  
  inflating: OpenNMT-py-1.2.0/config/config-transformer-base-4GPU.yml  
   creating: OpenNMT-py-1.2.0/data/
  inflating: OpenNMT-py-1.2.0/data/README.md  
  inflating: OpenNMT-py-1.2.0/data/ggnnsrc.txt  
  inflating: OpenNMT-py-1.2.0/data/ggnnsrcvocab.txt  
  inflating: OpenNMT-py-1.2.0/data/ggnntgt.txt  
  inflating: OpenNMT-py-1.2.0/data/ggnntgtv

--2021-01-30 08:39:28--  https://github.com/OpenNMT/OpenNMT-py/archive/1.2.0.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/OpenNMT/OpenNMT-py/zip/1.2.0 [following]
--2021-01-30 08:39:28--  https://codeload.github.com/OpenNMT/OpenNMT-py/zip/1.2.0
Resolving codeload.github.com (codeload.github.com)... 140.82.112.9
Connecting to codeload.github.com (codeload.github.com)|140.82.112.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35582731 (34M) [application/zip]
Saving to: ‘1.2.0.zip’

     0K .......... .......... .......... .......... ..........  0% 1001K 35s
    50K .......... .......... .......... .......... ..........  0% 1.94M 26s
   100K .......... .......... .......... .......... ..........  0% 1.94M 23s
   150K .......... .......... .......... .......... ..........  0% 40.9M 18s
   200K ....

# 対訳データのインストールと前処理

## 田中コーパス
田中コーパスというデータで翻訳実験を行います。日本の大学生が翻訳したデータです。そのうち適当な長さの5万文だけを抽出して実験しやすいようにしてくれたデータを用います。
train.{ja,en}というのが学習データ、dev.{ja,en}というのが開発データ、test.{ja,en}というのがテストデータです。

In [6]:
cd OpenNMT-py-1.2.0/

/content/OpenNMT-py-1.2.0


In [7]:
!git clone https://github.com/odashi/small_parallel_enja 

Cloning into 'small_parallel_enja'...
remote: Enumerating objects: 35, done.
remote: Total 35 (delta 0), reused 0 (delta 0), pack-reused 35
Unpacking objects: 100% (35/35), done.


In [8]:
ls small_parallel_enja

dev.en     train.en.000       train.en.vocab.all  train.ja.004
dev.ja     train.en.001       train.ja            train.ja.vocab.4k
README.md  train.en.002       train.ja.000        train.ja.vocab.all
test.en    train.en.003       train.ja.001
test.ja    train.en.004       train.ja.002
train.en   train.en.vocab.4k  train.ja.003


## データの前処理
日本語の単語分割と英語のtokenizationは済みです。onmt_preprocessというコマンドでデータの前処理をします。語彙テーブルを作り訓練データをOpenNMTように加工します。

In [9]:
!onmt_preprocess -train_src small_parallel_enja/train.en -train_tgt small_parallel_enja/train.ja -valid_src small_parallel_enja/dev.en -valid_tgt small_parallel_enja/dev.ja -save_data data/tanaka

[2021-01-30 08:39:38,656 INFO] Extracting features...
[2021-01-30 08:39:38,656 INFO]  * number of source features: 0.
[2021-01-30 08:39:38,656 INFO]  * number of target features: 0.
[2021-01-30 08:39:38,656 INFO] Building `Fields` object...
[2021-01-30 08:39:38,656 INFO] Building & saving training data...
[2021-01-30 08:39:38,715 INFO] Building shard 0.
[2021-01-30 08:39:40,565 INFO]  * saving 0th train data shard to data/tanaka.train.0.pt.
[2021-01-30 08:39:41,617 INFO]  * tgt vocab size: 8778.
[2021-01-30 08:39:41,624 INFO]  * src vocab size: 6636.
[2021-01-30 08:39:41,684 INFO] Building & saving validation data...
[2021-01-30 08:39:41,712 INFO] Building shard 0.
[2021-01-30 08:39:41,720 INFO]  * saving 0th valid data shard to data/tanaka.valid.0.pt.


In [10]:
ls data/

ggnnsrc.txt       morph/         src-val.txt        test_model2.src
ggnnsrcvocab.txt  README.md      tanaka.train.0.pt  test_model2.tgt
ggnntgt.txt       src-test.txt   tanaka.valid.0.pt  tgt-train.txt
ggnntgtvocab.txt  src-train.txt  tanaka.vocab.pt    tgt-val.txt


# 注意型NMT

## 学習
onmt_train というコマンドで学習します。いくつかオプションがあるので、変えてみると挙動も変わります。
--world_size で使う GPU の個数、--gpu_ranks で使う GPU を指定します（0スタートです）。指定しないと CPU になってしまいます。CPU は GPU の数十倍遅いです。
--train_steps は学習するステップを指定します。20,000だとこのデータは20分くらいで学習できます。

In [11]:
!onmt_train --help

usage: onmt_train [-h] [-config CONFIG] [-save_config SAVE_CONFIG]
                  [--src_word_vec_size SRC_WORD_VEC_SIZE]
                  [--tgt_word_vec_size TGT_WORD_VEC_SIZE]
                  [--word_vec_size WORD_VEC_SIZE] [--share_decoder_embeddings]
                  [--share_embeddings] [--position_encoding]
                  [--feat_merge {concat,sum,mlp}]
                  [--feat_vec_size FEAT_VEC_SIZE]
                  [--feat_vec_exponent FEAT_VEC_EXPONENT]
                  [--model_type {text,img,audio,vec}]
                  [--model_dtype {fp32,fp16}]
                  [--encoder_type {rnn,brnn,ggnn,mean,transformer,cnn}]
                  [--decoder_type {rnn,transformer,cnn}] [--layers LAYERS]
                  [--enc_layers ENC_LAYERS] [--dec_layers DEC_LAYERS]
                  [--rnn_size RNN_SIZE] [--enc_rnn_size ENC_RNN_SIZE]
                  [--dec_rnn_size DEC_RNN_SIZE]
                  [--audio_enc_pooling AUDIO_ENC_POOLING]
                  [--cnn_k

In [12]:
!onmt_train --data data/tanaka --save_model tanaka-model --world_size 1 --gpu_ranks 0 --train_steps 20000

[2021-01-30 08:40:31,070 INFO]  * src vocab size = 6636
[2021-01-30 08:40:31,071 INFO]  * tgt vocab size = 8778
[2021-01-30 08:40:31,071 INFO] Building model...
[2021-01-30 08:40:41,276 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(6636, 500, padding_idx=1)
        )
      )
    )
    (rnn): LSTM(500, 500, num_layers=2, dropout=0.3)
  )
  (decoder): InputFeedRNNDecoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(8778, 500, padding_idx=1)
        )
      )
    )
    (dropout): Dropout(p=0.3, inplace=False)
    (rnn): StackedLSTM(
      (dropout): Dropout(p=0.3, inplace=False)
      (layers): ModuleList(
        (0): LSTMCell(1000, 500)
        (1): LSTMCell(500, 500)
      )
    )
    (attn): GlobalAttention(
      (linear_in): Linear(in_features=500, out_features=500, bias=False)
      (li

## 翻訳モデル
学習されたモデルは tanaka-model_step_20000.pt のような名前で保存されています。

In [13]:
ls -l

total 343884
drwxr-xr-x  2 root root     4096 Aug 17 15:26 available_models/
drwxr-xr-x  4 root root     4096 Jan 30 08:39 build/
-rw-r--r--  1 root root     7873 Aug 17 15:26 CHANGELOG.md
drwxr-xr-x  2 root root     4096 Aug 17 15:26 config/
-rw-r--r--  1 root root     4126 Aug 17 15:26 CONTRIBUTING.md
drwxr-xr-x  3 root root     4096 Jan 30 08:39 data/
drwxr-xr-x  2 root root     4096 Jan 30 08:39 dist/
drwxr-xr-x  3 root root     4096 Aug 17 15:26 docs/
-rw-r--r--  1 root root       36 Aug 17 15:26 floyd_requirements.txt
-rw-r--r--  1 root root       30 Aug 17 15:26 floyd.yml
-rw-r--r--  1 root root     4408 Aug 17 15:26 github_deploy_key_opennmt_opennmt_py.enc
-rw-r--r--  1 root root     1072 Aug 17 15:26 LICENSE.md
drwxr-xr-x 11 root root     4096 Aug 17 15:26 onmt/
drwxr-xr-x  2 root root     4096 Jan 30 08:39 OpenNMT_py.egg-info/
-rw-r--r--  1 root root       99 Aug 17 15:26 preprocess.py
-rw-r--r--  1 root root     7867 Aug 17 15:26 README.md
-rw-r--r--  1 root root      267 Au

## デコード
onmt_translate というコマンドでデコードします。一つ前に学習した翻訳モデルのうちの1つを指定します。
--output pred.txt というので結果を保存するファイルを指定します。
--replace_unk というのは未知語処理をするというオプションです。
--verbose は結果を冗長に画面にも表示させるオプションです。

In [14]:
!onmt_translate --model tanaka-model_step_20000.pt --src small_parallel_enja/test.en --output pred.txt --replace_unk --verbose

[2021-01-30 08:58:22,099 INFO] Translating shard 0.
/usr/local/lib/python3.6/dist-packages/OpenNMT_py-1.2.0-py3.6.egg/onmt/translate/beam_search.py:209: UserWarning: An output with one or more elements was resized since it had shape [150], which does not match the required output shape [30, 5].This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  /pytorch/aten/src/ATen/native/Resize.cpp:19.)
  torch.mul(self.topk_scores, length_penalty, out=self.topk_log_probs)
[2021-01-30 08:58:23,099 INFO] 
SENT 1: ['they', 'finally', 'acknowledged', 'it', 'as', 'true', '.']
PRED 1: 彼 ら は ついに それ を 事実 と し た 。
PRED SCORE: -1.6511

[2021-01-30 08:58:23,099 INFO] 
SENT 2: ['he', 'didn', "'t", 'care', 'for', 'swimming', '.']
PRED 2: 彼 は 泳 ぐ 気 が し な かっ た 。
PRED SCORE: -0.8452

[2021-01-30 08:58:23,099 INFO] 
SENT 3

##BLEU による評価
正解（参照訳）のファイルを第一引数に、システムの出力をリダイレクト（<）で渡します。

In [15]:
!perl tools/multi-bleu.perl small_parallel_enja/test.ja < pred.txt

BLEU = 36.74, 66.4/44.9/31.6/23.0 (BP=0.957, ratio=0.958, hyp_len=5397, ref_len=5635)


In [16]:
cat pred.txt

彼 ら は ついに それ を 事実 と し た 。
彼 は 泳 ぐ 気 が し な かっ た 。
彼 は 姉 に 劣 ら ず 頭 が い い 。
１０ 時 前 に 戻 ら な けれ ば な ら な い 。
働 き 過ぎ た 。
彼女 は 私 達 の 隣 に 住 ん で い る 。
お 答え 下さ い 。
私 は 今 の ところ に 住 ん で い る 人 で す 。
この 試合 は 間違い だ 。
この 説明 を し て くれ ま せ ん か 。
彼女 は 先生 が 好き で す 。
商売 は 商売 で す 。
この 靴 は あなた に 二 年間 あ る だ ろ う 。
彼 は 息子 に 会 で その 会合 に 出席 し た 。
あなた の 仲間 に 行 く よう に し なさ い 。
仕事 の 終わり に 働 く 。
私 は 彼女 の 顔 を まとも に 見 る こと が でき な かっ た 。
日曜 の 朝教会 に は 誰 も い る 。
時計 を ２ 分 後 に 出 し ま す 。
あれ が 昨日 私 が 会 っ た 少年 で す 。
１０ 年 と は 待 つ の は 長 い 時間 だ 。
私 は 偶然 彼 に 東京 で 会 っ た 。
私 は よく 川 で 泳ぎ に 行 き ま す 。
彼 は 大声 で 助け を 求め た 。
彼 は その 子 を 火 から 救 っ た 。
私 は 彼 の 提案 に 同意 でき な い 。
私 たち は 何 度 も その 手紙 を 読 ん だ 。
私 は あなた に 会え て うれし い 。
お 父 さん に お 手伝い くださ い 。
彼 ら は その その 提案 を 断念 し た 。
私 は 特別 な こと が い い 。
それ 以来 彼 に は 誰 も 見 な かっ た 。
彼女 は 昔 、 羽振り が よ かっ た 。
それ を ほか の 少年 たち に 回 し て 下さ い 。
私 は 彼 の 名前 を 心 に 銘記 し て い る 。
私 は あなた と 結婚 し た い 。
今 すぐ 彼 ら を 見かけ る だ ろ う 。
地球 は 強 く な っ た 。
どちら が 欲し い か 私 に 教え て 下さ い 。
彼 が どこ に い たら よ かっ た か なあ 。
私 達 は 先週 数学 の 試験 を 受け

## 宿題1
デコード時のbeam幅を変更し翻訳性能の変化を確認してください。

In [ ]:
!onmt_translate --model tanaka-model_step_20000.pt --src small_parallel_enja/test.en --output pred.txt --replace_unk --verbose

## 宿題2
学習時のパラメータを変更し翻訳性能の改善を試してみてください。

# 自己注意型NMT（transformer）

## 学習
onmt_train というコマンドでtransformerを指定して学習します。
--world_sizeで使う GPU の個数を1に、--gpu_ranksで使う GPUを0指定します。
--encoder_type transformerと--decoder_type transformerでエンコーダとデコーダの種類をtransformerに指定します。
-position_encodingでposition encodingと指定します。
--train_steps は学習するステップを指定します。20,000だとこのデータは40分くらいで学習できます。
他のパラメータはtransformerの性能を出すためのディフォルト設定です。

In [ ]:
!onmt_train --data data/tanaka --save_model tanaka-model-transformer  \
        -layers 6 -rnn_size 512 -word_vec_size 512 -transformer_ff 2048 -heads 8  \
        -encoder_type transformer -decoder_type transformer -position_encoding \
        -train_steps 20000  -max_generator_batches 2 -dropout 0.1 \
        -batch_type tokens -normalization tokens  -accum_count 2 \
        -optim adam -adam_beta2 0.998 -decay_method noam -warmup_steps 8000 -learning_rate 2 \
        -max_grad_norm 0 -param_init 0  -param_init_glorot \
        -label_smoothing 0.1 -save_checkpoint_steps 10000 \
        -world_size 1 -gpu_ranks 0 

## 翻訳モデル
学習されたモデルは tanaka-model-transformer_step_20000.pt のような名前で保存されています。

In [ ]:
ls -l

## デコード
onmt_translate というスクリプトでデコードします。一つ前に学習した翻訳モデルのうちの1つを指定します。
--output pred.txt というので結果を保存するファイルを指定します。
--replace_unk というのは未知語処理をするというオプションです。
--verbose は結果を冗長に画面にも表示させるオプションです。

In [ ]:
!onmt_translate --model tanaka-model-transformer_step_20000.pt --src small_parallel_enja/test.en --output pred-transformer.txt --replace_unk --verbose

##BLEU による評価
正解（参照訳）のファイルを第一引数に、システムの出力をリダイレクト（<）で渡します。

In [ ]:
!perl tools/multi-bleu.perl small_parallel_enja/test.ja < pred-transformer.txt

性能出ない理由は学習が足りないからです。-batch_type tokensデフォルト-batch_size 64 は1 step 64 tokenつまり4文ぐらいで、20000 stepですと4×20000/50000=1.6 epochしか学習していません。一方RNNの場合は1 step 64文で、20000 stepですと64×20000/50000=25.6 epoch学習しています。

In [ ]:
cat pred-transformer.txt